In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import torch
import torchvision
from torch.utils.data import Dataset
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')
from pipe_dataset import pipedataset
from torch.utils.data import DataLoader


In [ ]:
device = torch.device('cuda'if torch.cuda.is_available() else 'cpu')

In [ ]:
from torch.utils.data import random_split

In [ ]:
def load_checkpoint(checkpoint, model):
  print("=Load checkpoint")
  model.load_state_dict(checkpoint["state_dict"])

In [ ]:
def save_checkpoints(state, filename="check_point.pth.tar"):
  print("=saving checkpoint")
  torch.save(state, filename)





In [1]:
def get_loaders(
    img_dir,
    mask_dir,
    batch_size,
    num_workers=4,
    pin_memory=True
):
    pytorch_dataset = pipedataset(img_dir, mask_dir, transform=None)
    generator = torch.Generator().manual_seed(42)
    train_ds, val_ds, test_ds = random_split(pytorch_dataset, [0.7, 0.2, 0.1], generator=generator)
    #train_ds, val_ds, test_ds = random_split(pytorch_dataset, [len(pytorch_dataset)*0.7, len(pytorch_dataset)*0.2, len(pytorch_dataset)*0.1])

    train_data_loader = DataLoader(
        train_ds,
        batch_size = batch_size,
        num_workers=num_workers,
        pin_memory = True,
        shuffle =True,
    )

    val_data_loader = DataLoader(
        val_ds,
        batch_size = batch_size,
        num_workers = num_workers,
        pin_memory=True,
        shuffle=False,
    )
    return train_data_loader, val_data_loader

In [ ]:
def check_accuracy(loader, model, device=device):
  num_correct = 0
  num_pixel = 0
  model.eval()
  dice_score = 0
  with torch.no_grad():
    for x, y in loader:
      x = x.to(device)
      y = y.to(device)
      preds = torch.sigmoid(model(x))
      preds = (preds >0.5).float()
      num_corrects += (pred ==y).sum()
      num_pioxels += torch.numel(pred)
      dice_score += (2 *(preds*y).sum())/((preds+y).sum() +1e-8)

  print(f"Dice_score:{dice_score/len(loader)}")
  model.train()

